In [7]:
import pandas as pd #to set Mars facts into a db

from bs4 import BeautifulSoup as bs # "You didn't write that awful page. You're just trying to get some data out of it."
#--------------------------------------Used to navigate/pull data out of html and xml files

from splinter import Browser #abstraction layer on top of selenium

from selenium import webdriver #The selenium.webdriver module provides all the WebDriver implementations.
#-------------------------------Currently supported WebDriver implementations are Firefox, Chrome, IE and Remote.

import time #for browser delays

In [9]:
#instatiate the Browser class, should open up a testable browser window
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path, headless=False) 

## scrape the NASA news site

In [10]:
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)

In [14]:
#use the html attribute to get the html content of the visited page:
html = browser.html

#make the soup using the built-in python parser
soup = bs(html,'html.parser')

#assign the scraped data into variables
news_title = soup.find('ul', class_="item_list").find('li',class_="slide").find('div',class_="content_title").text
print(news_title)

AI Is Helping Scientists Discover Fresh Craters on Mars


In [15]:
#assign the scraped data into variables
news_teaser = soup.find('ul',class_="item_list").find('li',class_="slide").find('div',class_="article_teaser_body").text
print(news_teaser)

It's the first time machine learning has been used to find previously unknown craters on the Red Planet.


## Scrape the featured image

In [16]:
browser.visit('https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars')

In [17]:
#Make sure to find the image url to the full size .jpg image
browser.find_by_id('full_image').click()

In [18]:
#pause to allow for the user to 'see' something happening
time.sleep(1)

browser.links.find_by_partial_text('more info').click()

In [19]:
#use the html attribute to get the html content of the visited page:
html = browser.html

#make the soup using the built-in python parser
soup = bs(html,'html.parser')

#assign the scraped url
featured_img_url = soup.find('figure', class_='lede').a['href']
print(featured_img_url)

/spaceimages/images/largesize/PIA18273_hires.jpg


## Scrape Facts

In [20]:
url = 'https://space-facts.com/mars/'
browser.visit(url)

In [21]:
# pandas... pandas, pandas, pandas!
facts_df = pd.read_html(url)[0]
facts_df.columns = ["Description","Mars"]
facts_df

,Description,Mars
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [23]:
#rearrange
facts = facts_df.transpose()
facts

,0,1,2,3,4,5,6,7,8
Description,Equatorial Diameter:,Polar Diameter:,Mass:,Moons:,Orbit Distance:,Orbit Period:,Surface Temperature:,First Record:,Recorded By:
Mars,"6,792 km","6,752 km",6.39 × 10^23 kg (0.11 Earths),2 (Phobos & Deimos),"227,943,824 km (1.38 AU)",687 days (1.9 years),-87 to -5 °C,2nd millennium BC,Egyptian astronomers


In [25]:
#render df to html
facts_table = facts.to_html('MarsFacts.html',index = False)
facts_table

## Scrape the hemispheres

In [26]:
browser.visit('https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars')

In [27]:
hemisphere_img_urls = []

#use the html attribute to get the html content of the visited page:
html = browser.html

#make the soup using the built-in python parser
soup = bs(html,'html.parser')

In [28]:
links = soup.find('div', class_='collapsible results').find_all('div',class_='item')
links

[<div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png"/></a><div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p></div> <!-- end description --></div>,
 <div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><img alt="Schiaparelli Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/08eac6e22c07fb1fe72223a79252de20_schiapa

In [34]:
#create a loop to select each link
for link in links:
    hem_title = link.find('div', class_='description').find('a', class_='itemLink product-item').h3.text
#print(hem_title)
    hem_url = 'https://astrogeology.usgs.gov'
    each_hem_image_url = hem_url + link.find('a',class_='itemLink product-item')['href']
    browser.visit(each_hem_image_url)
    html = browser.html
    soup = bs(html, 'html.parser')
    full_image_url = soup.find('div',class_='downloads').a['href']
    each_hemisphere_image = {
        "title" : hem_title,
        "image_url" : full_image_url
    } 
    hemisphere_img_urls.append(each_hemisphere_image)

{'title': 'Cerberus Hemisphere Enhanced', 'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}
{'title': 'Schiaparelli Hemisphere Enhanced', 'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}
{'title': 'Syrtis Major Hemisphere Enhanced', 'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}
{'title': 'Valles Marineris Hemisphere Enhanced', 'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}


In [36]:
hemisphere_img_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [37]:
browser.quit()